In [1]:
# Java
!apt-get update # Обновляем список пакетов в системе

!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Устанавливаем OpenJDK 8 (Java Development Kit) без графического интерфейса
# > /dev/null: Перенаправляет вывод команды в /dev/null (пустое устройство), чтобы скрыть вывод в консоли

# Spark
!wget https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Загружаем архив Spark 3.1.1, предварительно скомпилированный для Hadoop 3.2, с сайта Apache
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Распаковываем архив

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
--2025-04-18 08:56:57--  https://archive.apache.org/dist/spark/spark-3

In [2]:
# настраеваем переменные окружения
import os  # Импортирует модуль для работы с операционной системой

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"  # Устанавливает переменную окружения JAVA_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2" # Устанавливает переменную окружения SPARK_HOME

In [3]:
# Устанавливаем findspark
!pip install -q findspark
!pip install pyspark

import findspark
findspark.init() # Инициализируем findspark

**Проверка установки. Инициализация сессии, импорт необходимых библиотек**

In [4]:
# Создание SparkContext (проверка установки)
import re
from pyspark.sql.functions import udf, col, year, lit, when, explode, regexp_replace, lower, split, rank
from typing import List
# Создание SparkContext
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import max, sum, countDistinct, desc


spark = SparkSession \
    .builder \
    .appName("L2_reports_with_apache_spark") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0")\
    .getOrCreate()

sc = spark.sparkContext# Получаем SparkContext из SparkSession, SparkContext - это точка входа в Spark

print(sc.version)  # Выводим версию Spark

3.1.1


**Загрузка данных**

In [5]:
posts_data = spark.read\
.format('xml')\
.options(rowTag='row')\
.load('posts_sample.xml')
print("Posts")
posts_data.printSchema()

Posts
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



# Задание

Задание: Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

In [6]:
# Считываем programming-languages.csv
langs_df = spark.read.option("header", True).csv("programming-languages.csv")
langs_list = [row["name"].lower() for row in langs_df.select("name").distinct().collect()]# Преобразуем названия языков в список
langs_broadcast = spark.sparkContext.broadcast(langs_list)# Распространяем список языков на все узлы кластера


In [7]:
# Обработка тегов и извлечение года
# 1. Удаляем угловые скобки из тегов и разделяем строку тегов в массив
# 2. Извлекаем год из даты последней активности поста
# 3. Выбираем только нужные столбцы и переименовываем _ViewCount
posts_data = posts_data.withColumn("tags", split(regexp_replace(col("_Tags"), "[<>]", ","), ",")) \
                   .withColumn("year", year(col("_LastActivityDate"))) \
                   .select("tags", "year", "_ViewCount") \
                   .withColumnRenamed("_ViewCount", "views")

In [8]:
# Разворачивание массива тегов в отдельные строки
tags_df = posts_data.withColumn("tag", explode(col("tags"))).select("tag", "year", "views")
filtered_tags = tags_df.filter(lower(col("tag")).isin(langs_broadcast.value))# Фильтрация по языкам программирования

# Группировка по году и языку и считаем суммарное количество просмотров
views_by_lang = filtered_tags.groupBy("year", "tag").agg(sum("views").alias("total_views")) \
                             .filter((col("year") >= 2010) & (col("year") <= 2020))


In [9]:
# Добавление ранга в пределах каждого года — 1 место имеет больше всего просмотров
ranking = views_by_lang.withColumn(
    "rank",
    rank().over(Window.partitionBy("year").orderBy(col("total_views").desc()))
)

# Выбираем 10 самых используемых языков на каждый год
top_10_by_year = ranking.filter(col("rank") <= 10) \
                        .select("year", "tag", "total_views") \
                        .orderBy("year", col("total_views").desc())

# Сохраняем итоговый DataFrame в формате Parquet
top_10_by_year.write.mode("overwrite").parquet("popular_languages_top10.parquet")

# Отображаем результат — топ-10 языков для каждого года
top_10_by_year.show(50, truncate=False)


+----+-----------+-----------+
|year|tag        |total_views|
+----+-----------+-----------+
|2010|java       |53333      |
|2010|matlab     |51865      |
|2010|objective-c|43878      |
|2010|php        |39730      |
|2010|javascript |37059      |
|2010|python     |25930      |
|2010|ruby       |15864      |
|2010|c          |13810      |
|2010|delphi     |7680       |
|2010|r          |7499       |
|2011|java       |121315     |
|2011|python     |89637      |
|2011|c          |73116      |
|2011|php        |67341      |
|2011|javascript |61631      |
|2011|objective-c|54815      |
|2011|r          |14394      |
|2011|ruby       |9771       |
|2011|cython     |8109       |
|2011|delphi     |6724       |
|2012|php        |303789     |
|2012|java       |272219     |
|2012|python     |220014     |
|2012|javascript |204780     |
|2012|objective-c|50825      |
|2012|bash       |36183      |
|2012|ruby       |32695      |
|2012|c          |31552      |
|2012|delphi     |16898      |
|2012|lu